In [1]:
import gym
import random
import numpy as np
import tensorflow as tf
from collections import deque
from skimage.color import rgb2gray
from skimage.transform import resize
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, Dense, Flatten, Lambda, merge
from keras.layers.convolutional import Conv2D
from keras import backend as K

/home/dtkachenko/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# The Deterministic-v4 version of the Atari environments is exactly equivalent to what DeepMind used in their paper.
# Specifically, "-v4" essentially implements frame skipping so that the agent acts on every 4th frame. 
# This reduces the number of frames to process and thus reduces training time 
# without sacrificing agent's game performance.
env = gym.make('BreakoutDeterministic-v4')

In [3]:
EPISODES = 50000
MODEL_PATH = "model/breakout_dueling_ddqn.h5"
TENSORBOARD_LOG_PATH = 'summary/breakout_dueling_ddqn'
MODEL_CHECKPOINT_FREQUENCY = 100

# actions:
NO_ACTION_NO_BALL = 0
NO_ACTION = 1
MOVE_RIGHT = 2
MOVE_LEFT = 3

In [4]:
# 210x160x3 (RGB color) -> 84x84 (grayscale)
# also transforms floats to 8-bit ints to conserve memory
def preprocess(observed_state):
    return np.uint8(resize(rgb2gray(observed_state), (84, 84), mode='constant') * 255)

In [5]:
def get_initial_state_history(observed_state):
    # No preceding frames at the start of an episode, so just copy 4 times
    state = preprocess(observed_state)
    history = np.stack((state, state, state, state), axis=2)
    history = np.reshape([history], (1, 84, 84, 4))
    return state, history

In [ ]:
class DuelingDDQNAgent:
    def __init__(self, action_size):
        self.render = False
        self.load_model = False

        # environment settings
        self.state_size = (84, 84, 4)
        self.action_size = action_size

        # epsilon-greedy policy parameters
        self.epsilon = 1.
        self.epsilon_start, self.epsilon_end = 1.0, 0.1
        self.exploration_steps = 1000000.
        self.epsilon_decay_step = (self.epsilon_start - self.epsilon_end) / self.exploration_steps

        # training parameters
        self.batch_size = 32
        self.train_start = 50000
        self.update_target_rate = 10000
        self.discount_factor = 0.99
        self.memory = deque(maxlen=400000)
        self.no_op_steps = 30

        # build
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

        self.optimizer = self.optimizer()

        self.sess = tf.InteractiveSession()
        K.set_session(self.sess)

        self.avg_q_max, self.avg_loss = 0, 0
        self.summary_placeholders, self.update_ops, self.summary_op = self.setup_summary()
        self.summary_writer = tf.summary.FileWriter(TENSORBOARD_LOG_PATH, self.sess.graph)
        self.sess.run(tf.global_variables_initializer())

        if self.load_model:
            self.model.load_weights(MODEL_PATH)

    # if error is in [-1; 1] cost is quadratic to error, otherwise – linear
    def optimizer(self):
        a = K.placeholder(shape=(None,), dtype='int32')
        y = K.placeholder(shape=(None,), dtype='float32')

        py_x = self.model.output

        a_one_hot = K.one_hot(a, self.action_size)
        q_value = K.sum(py_x * a_one_hot, axis=1)
        error = K.abs(y - q_value)

        quadratic_part = K.clip(error, 0.0, 1.0)
        linear_part = error - quadratic_part
        loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)

        optimizer = RMSprop(lr=0.00025, epsilon=0.01)
        updates = optimizer.get_updates(self.model.trainable_weights, [], loss)
        train = K.function([self.model.input, a, y], [loss], updates=updates)

        return train

    # frame representation -> CNN -> advantage/value streams -> merge two streams -> Q-value of each action
    def build_model(self):
        input = Input(shape=self.state_size)
        shared = Conv2D(32, (8, 8), strides=(4, 4), activation='relu')(input)
        shared = Conv2D(64, (4, 4), strides=(2, 2), activation='relu')(shared)
        shared = Conv2D(64, (3, 3), strides=(1, 1), activation='relu')(shared)
        flatten = Flatten()(shared)

        # stream that estimates state-dependent action advantages
        advantage_fc = Dense(512, activation='relu')(flatten)
        advantage = Dense(self.action_size)(advantage_fc)
        advantage = Lambda(lambda a: a[:, :] - K.mean(a[:, :], keepdims=True),
                           output_shape=(self.action_size,))(advantage)

        # stream which estimates state values
        value_fc = Dense(512, activation='relu')(flatten)
        value = Dense(1)(value_fc)
        value = Lambda(lambda s: K.expand_dims(s[:, 0], -1),
                       output_shape=(self.action_size,))(value)

        # merge two streams to produce Q-value
        q_value = merge([value, advantage], mode='sum')
        model = Model(inputs=input, outputs=q_value)
        model.summary()

        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # epsilon-greedy policy
    def get_action(self, history):
        history = np.float32(history / 255.0)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(history)
            return np.argmax(q_value[0])

    # save sample <s, a, r, s'> to the replay memory
    def replay_memory(self, history, action, reward, next_history, dead):
        self.memory.append((history, action, reward, next_history, dead))

    # sample from replay memory (batches of specified size)
    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        if self.epsilon > self.epsilon_end:
            self.epsilon -= self.epsilon_decay_step

        mini_batch = random.sample(self.memory, self.batch_size)

        history = np.zeros((self.batch_size, self.state_size[0],
                            self.state_size[1], self.state_size[2]))
        next_history = np.zeros((self.batch_size, self.state_size[0],
                                 self.state_size[1], self.state_size[2]))
        target = np.zeros((self.batch_size,))
        action, reward, dead = [], [], []

        for i in range(self.batch_size):
            history[i] = np.float32(mini_batch[i][0] / 255.)
            next_history[i] = np.float32(mini_batch[i][3] / 255.)
            action.append(mini_batch[i][1])
            reward.append(mini_batch[i][2])
            dead.append(mini_batch[i][4])

        value = self.model.predict(history)
        target_value = self.target_model.predict(next_history)

        # get max Q-value at s' from target model
        for i in range(self.batch_size):
            if dead[i]:
                target[i] = reward[i]
            else:
                # key DDQN idea: select action from model, update value from target model
                target[i] = reward[i] + self.discount_factor * target_value[i][np.argmax(value[i])]

        loss = self.optimizer([history, action, target])
        self.avg_loss += loss[0]

    def setup_summary(self):
        episode_total_reward = tf.Variable(0.)
        episode_avg_max_q = tf.Variable(0.)
        episode_duration = tf.Variable(0.)
        episode_avg_loss = tf.Variable(0.)

        tf.summary.scalar('Total Reward/Episode', episode_total_reward)
        tf.summary.scalar('Average Max Q/Episode', episode_avg_max_q)
        tf.summary.scalar('Duration/Episode', episode_duration)
        tf.summary.scalar('Average Loss/Episode', episode_avg_loss)

        summary_vars = [episode_total_reward, episode_avg_max_q,
                        episode_duration, episode_avg_loss]
        summary_placeholders = [tf.placeholder(tf.float32) for _ in
                                range(len(summary_vars))]
        update_ops = [summary_vars[i].assign(summary_placeholders[i]) for i in
                      range(len(summary_vars))]
        summary_op = tf.summary.merge_all()
        return summary_placeholders, update_ops, summary_op

In [ ]:
agent = DuelingDDQNAgent(action_size=3)

scores, episodes, global_step = [], [], 0

for e in range(EPISODES):
    done = False
    dead = False
    # 1 episode = 5 lives
    step, score, start_life = 0, 0, 5
    observed_state = env.reset()

    # One of the ideas from DeepMind's papers: don't take any action for
    # some random number of steps at the start of an episode
    for _ in range(random.randint(1, agent.no_op_steps)):
        observed_state, _, _, _ = env.step(NO_ACTION)

    state, history = get_initial_state_history(observed_state)

    while not done:
        if agent.render:
            env.render()
        global_step += 1
        step += 1

        # get action for the current history
        action = agent.get_action(history)

        # model returns one of the three actions (from [0; 2]),
        # but 'real' actions are 1 through 3, because 0 is no-op (no action, no ball)
        real_action = action + 1

        observed_state, reward, done, info = env.step(real_action)

        # build next state / history
        next_state = preprocess(observed_state)
        next_state = np.reshape([next_state], (1, 84, 84, 1))
        next_history = np.append(next_state, history[:, :, :, :3], axis=3)

        agent.avg_q_max += np.amax(
            agent.model.predict(np.float32(history / 255.))[0])

        # check if the agent is 'dead' and update remaining lives
        if start_life > info['ale.lives']:
            dead = True
            start_life = info['ale.lives']

        reward = np.clip(reward, -1., 1.)

        # save the sample <s, a, r, s', d> to the replay memory and train model
        agent.replay_memory(history, action, reward, next_history, dead)
        agent.train_replay()

        # every once in a while, update the target model
        if global_step % agent.update_target_rate == 0:
            agent.update_target_model()

        score += reward

        if dead:
            dead = False

            # reset history
            _, history = get_initial_state_history(observed_state)
        else:
            history = next_history

        # log progress to console and TensorBoard on episode end
        if done:
            if global_step > agent.train_start:
                stats = [score, agent.avg_q_max / float(step), step, agent.avg_loss / float(step)]
                for i in range(len(stats)):
                    agent.sess.run(agent.update_ops[i], feed_dict={
                        agent.summary_placeholders[i]: float(stats[i])
                    })
                summary_str = agent.sess.run(agent.summary_op)
                agent.summary_writer.add_summary(summary_str, e + 1)

            print("episode:", e, "  score:", score, "  memory length:",
                    len(agent.memory), "  epsilon:", agent.epsilon,
                    "  global_step:", global_step, "  average_q:",
                    agent.avg_q_max / float(step), "  average loss:",
                    agent.avg_loss / float(step))

            agent.avg_q_max, agent.avg_loss = 0, 0

    if e % MODEL_CHECKPOINT_FREQUENCY == 0:
        agent.model.save_weights(MODEL_PATH)

/home/dtkachenko/.local/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/dtkachenko/.local/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 84, 84, 4)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 20, 32)   8224        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 9, 9, 64)     32832       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 7, 7, 64)     36928       conv2d_2[0][0]                   
__________________________________________________________________________________________________
flatten_1 

episode: 15   score: 1.0   memory length: 2935   epsilon: 1.0   global_step: 2935   average_q: 0.01634719452775288   average loss: 0.0
episode: 16   score: 1.0   memory length: 3103   epsilon: 1.0   global_step: 3103   average_q: 0.01888675812543148   average loss: 0.0
episode: 17   score: 1.0   memory length: 3251   epsilon: 1.0   global_step: 3251   average_q: 0.01846491955094845   average loss: 0.0
episode: 18   score: 1.0   memory length: 3420   epsilon: 1.0   global_step: 3420   average_q: 0.018748351802619603   average loss: 0.0
episode: 19   score: 1.0   memory length: 3592   epsilon: 1.0   global_step: 3592   average_q: 0.017118361746069302   average loss: 0.0
episode: 20   score: 0.0   memory length: 3714   epsilon: 1.0   global_step: 3714   average_q: 0.01637965846104456   average loss: 0.0
episode: 21   score: 2.0   memory length: 3904   epsilon: 1.0   global_step: 3904   average_q: 0.02081153277111681   average loss: 0.0
episode: 22   score: 0.0   memory length: 4010   epsi

episode: 76   score: 2.0   memory length: 12804   epsilon: 1.0   global_step: 12804   average_q: 0.020354713350965108   average loss: 0.0
episode: 77   score: 0.0   memory length: 12897   epsilon: 1.0   global_step: 12897   average_q: 0.015966180822141067   average loss: 0.0
episode: 78   score: 0.0   memory length: 13003   epsilon: 1.0   global_step: 13003   average_q: 0.015659592070458632   average loss: 0.0
episode: 79   score: 1.0   memory length: 13150   epsilon: 1.0   global_step: 13150   average_q: 0.020116582392480504   average loss: 0.0
episode: 80   score: 1.0   memory length: 13295   epsilon: 1.0   global_step: 13295   average_q: 0.019339052292293515   average loss: 0.0
episode: 81   score: 2.0   memory length: 13497   epsilon: 1.0   global_step: 13497   average_q: 0.020629665330236795   average loss: 0.0
episode: 82   score: 1.0   memory length: 13619   epsilon: 1.0   global_step: 13619   average_q: 0.018081292624539527   average loss: 0.0
episode: 83   score: 2.0   memory 

episode: 136   score: 1.0   memory length: 21831   epsilon: 1.0   global_step: 21831   average_q: 0.017232694335538767   average loss: 0.0
episode: 137   score: 1.0   memory length: 21987   epsilon: 1.0   global_step: 21987   average_q: 0.019207610569607753   average loss: 0.0
episode: 138   score: 0.0   memory length: 22111   epsilon: 1.0   global_step: 22111   average_q: 0.016169302030316284   average loss: 0.0
episode: 139   score: 0.0   memory length: 22252   epsilon: 1.0   global_step: 22252   average_q: 0.015783355888375577   average loss: 0.0
episode: 140   score: 1.0   memory length: 22422   epsilon: 1.0   global_step: 22422   average_q: 0.018725748316330067   average loss: 0.0
episode: 141   score: 0.0   memory length: 22543   epsilon: 1.0   global_step: 22543   average_q: 0.016336081526434617   average loss: 0.0
episode: 142   score: 0.0   memory length: 22669   epsilon: 1.0   global_step: 22669   average_q: 0.016110043233585735   average loss: 0.0
episode: 143   score: 2.0  

...

episode: 5158   score: 30.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1388680   average_q: 1.0472811028012075   average loss: 0.004025746445222467
episode: 5159   score: 7.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1388981   average_q: 1.213777364686478   average loss: 0.0039899254428915855
episode: 5160   score: 15.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1389603   average_q: 1.1653863897373438   average loss: 0.003964782182941022
episode: 5161   score: 18.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1390359   average_q: 1.1822432232675728   average loss: 0.004103009897301457
episode: 5162   score: 24.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1391262   average_q: 1.1107530758065374   average loss: 0.0037665061306121736
episode: 5163   score: 22.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1392087   average_q: 1.0453

episode: 5205   score: 18.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1424289   average_q: 1.0782079284792172   average loss: 0.004556993548706299
episode: 5206   score: 16.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1424936   average_q: 1.1975025901516216   average loss: 0.004343411922537602
episode: 5207   score: 24.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1425795   average_q: 1.0396200677811476   average loss: 0.004358322481186768
episode: 5208   score: 28.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1426791   average_q: 1.0991621561587814   average loss: 0.0042228365223948286
episode: 5209   score: 22.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1427632   average_q: 1.1148373073190627   average loss: 0.004429496698428663
episode: 5210   score: 15.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1428266   average_q: 1.167

episode: 5252   score: 22.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1461892   average_q: 1.0947438527795452   average loss: 0.0039424172178701995
episode: 5253   score: 19.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1462704   average_q: 1.0938344157755082   average loss: 0.004107477149069928
episode: 5254   score: 24.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1463658   average_q: 1.1630152782554146   average loss: 0.004027399449610196
episode: 5255   score: 22.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1464501   average_q: 1.138362000067746   average loss: 0.004219992161968177
episode: 5256   score: 24.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1465398   average_q: 1.0874079460882422   average loss: 0.003974181630284242
episode: 5257   score: 21.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1466168   average_q: 1.0804

episode: 5299   score: 29.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1503795   average_q: 1.1842304976227447   average loss: 0.003946431234619105
episode: 5300   score: 29.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1504861   average_q: 1.156602218677246   average loss: 0.003942696945623828
episode: 5301   score: 17.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1505513   average_q: 1.3074445031080508   average loss: 0.0036633974986623663
episode: 5302   score: 21.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1506322   average_q: 1.2638262006165217   average loss: 0.003650269692715336
episode: 5303   score: 29.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1507518   average_q: 1.1847566189773904   average loss: 0.0038039836685841337
episode: 5304   score: 26.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1508499   average_q: 1.154

episode: 5346   score: 22.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1542198   average_q: 1.167226966957117   average loss: 0.004019278773436478
episode: 5347   score: 20.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1542910   average_q: 1.2451661973145236   average loss: 0.003911542023641313
episode: 5348   score: 15.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1543513   average_q: 1.1588671348284727   average loss: 0.004093476955429071
episode: 5349   score: 18.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1544200   average_q: 1.223057627721298   average loss: 0.0038242736626945143
episode: 5350   score: 29.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1545315   average_q: 1.0573009040590893   average loss: 0.003852784317644572
episode: 5351   score: 18.0   memory length: 400000   epsilon: 0.09999910000958534   global_step: 1545961   average_q: 1.30303